In [ ]:
import sys
from pathlib import Path

# add directory with ribasim-repository
sys.path.append("..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping-repository
sys.path.append("..")
import ribasim_lumping

import pandas as pd
import geopandas as gpd

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", UserWarning)

import networkx as nx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
dhydro_dir = Path(base_dir, "d-hydro")
results_dir = Path(base_dir, "results")
network_name = "tki_bommelerwaard"

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden", "praktijkpeilen_bommelerwaard.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['code', 'geometry']]

In [ ]:
# Create networkanalysis
network = ribasim_lumping.create_ribasim_lumping_network(
    name=network_name, 
    dhydro_dir=dhydro_dir,
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)
# network.export_to_geopackage()

In [ ]:
# Add simulation sets for winter, summer, etc.
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=dhydro_dir,
    simulations_names=["tki_bommelerwaard"],
    simulation_output_dir="fm\\output",
    simulations_ts=[-1],
);

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data()
# network.export_to_geopackage()

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    orifices=True,
    structures_ids_to_include=[
        # 'DKR_009969', 'DKR_010011',
    ], 
    structures_ids_to_exclude=[
        'STW_001072B', 'STW_056492B'
    ], 
    edge_ids_to_include=[
    ],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

In [ ]:
# specify translation split_nodes to ribasim-nodes per type
split_node_type_conversion = {
    'weir': 'TabulatedRatingCurve', 
    'uniweir': 'TabulatedRatingCurve' ,
    'pump': 'Pump', 
    'culvert': 'TabulatedRatingCurve', 
    'manual': 'ManningResistance',
    'orifice' : 'TabulatedRatingCurve',
    'edge': 'ManningResistance',
}
# specify translation for specific split_nodes to ribasim-nodes
split_node_id_conversion = {
    # 'sto_AE80770024': 'ManningResistance', 
    # 'kdu_DR80740070': 'ManningResistance',
    # # duikers voor wijk Zuidwijken
    # 'kdu_DR80940046': 'ManningResistance', 
    # 'kdu_DR80950043': 'ManningResistance', 
    # 'kdu_DR80950151': 'ManningResistance', 
}

In [ ]:
# Generate Ribasim model en exporteer naar geopackage
ribasim_model = network.generate_ribasim_model(
    split_node_type_conversion=split_node_type_conversion, 
    split_node_id_conversion=split_node_id_conversion
)
ribasim_model.write(f"{results_dir}/{network.name}")